In [1]:
import numpy as np
import cv2
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import salientregions as sr

In [2]:
#Load the image
path_to_image = '/home/dafne/SalientRegions/LargeScaleImaging/Data/OxFrei/01_graffiti/JPG/_original.jpg'
#path_to_image = '../../../Data/OxFrei/09_small_palace/_original.jpg'
original = cv2.imread(path_to_image)
sr.show_image(original)
path_to_image = '/home/dafne/SalientRegions/LargeScaleImaging/Data/OxFrei/01_graffiti/JPG/viewpoint1.jpg'
viewpoint = cv2.imread(path_to_image)
#sr.show_image(viewpoint)

In [3]:
#Convert to grey scale
grayscale = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
grayscale1 = cv2.cvtColor(viewpoint, cv2.COLOR_BGR2GRAY)
sr.show_image(grayscale)
#sr.show_image(grayscale1)

In [ ]:
#Binarize with OTSU
#t_otsu, binarized = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
binarized = sr.OtsuBinarizer().binarize(grayscale)

#ret, binarized1 = cv2.threshold(grayscale1, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
sr.show_image(binarized)
#sr.show_image(binarized1)

In [ ]:
#Binarize according to threshold
thres = 128
#binarized = sr.binarize(grayscale, threshold=thres)
binarizer = sr.ThresholdBinarizer(thres)
binarized = binarizer.binarize(grayscale)
sr.show_image(binarized)

In [ ]:
img = grayscale
SE_size_factor = 0.02
lam_factor = 3
area_factor_large = 0.001
area_factor_verylarge = 0.1
SE, lam = sr.get_SE(img, SE_size_factor=SE_size_factor, lam_factor=lam_factor)
connectivity = 4
weights=(0.33,0.33,0.33)

print 'lam is: %i' %lam

In [ ]:
binarizer = sr.DatadrivenBinarizer(area_factor_large=area_factor_large, area_factor_verylarge=area_factor_verylarge, 
                                           lam=lam, weights=weights, connectivity=connectivity)
binarized = binarizer.binarize(grayscale)

In [ ]:
t, binarized_dd = sr.data_driven_binarization(img, area_factor_large=area_factor_large, area_factor_verylarge=area_factor_verylarge, 
                                           lam=lam, weights=weights, connectivity=connectivity, visualize=True)
print t
sr.show_image(binarized_dd)

In [4]:
regions_dmsr = sr.get_salient_regions(original, visualize=False)
sr.show_image(original)
_ = sr.visualize_elements(original, holes=regions_dmsr['holes'], islands=regions_dmsr['islands'],
                     indentations=regions_dmsr['indentations'], protrusions=regions_dmsr['protrusions'])

In [5]:
regions = sr.get_salient_regions_MSSR(grayscale, min_thres=80, max_thres=90)
_ = sr.visualize_elements(original, holes=regions['holes'], islands=regions['islands'],
                     indentations=regions['indentations'], protrusions=regions['protrusions'])

In [6]:
background = np.zeros(original.shape, dtype='uint8')
dmsr = sr.visualize_elements(background, holes=regions_dmsr['holes'], islands=regions_dmsr['islands'],
                     indentations=regions_dmsr['indentations'], protrusions=regions_dmsr['protrusions'])
mssr = sr.visualize_elements(background, holes=regions['holes'], islands=regions['islands'],
                     indentations=regions['indentations'], protrusions=regions['protrusions'])
sr.image_diff(dmsr, mssr)

False

In [ ]:
a = regions_dmsr['holes']
b = regions['holes']
sr.show_image(a)
sr.show_image(b)
sr.show_image(cv2.bitwise_and(a,b))
sr.image_diff(a, b)